# Quantlab Data Pipeline Demo

Use this notebook to manually exercise the pipeline functions. Ensure `credentials.yml` is present at the repo root with Alpha Vantage and WRDS credentials, and that dependencies (`pandas`, `pyarrow`, `requests`, `wrds`) are installed.

In [2]:
from datetime import date
import logging
import sys
from pathlib import Path

# Ensure repo root is on sys.path for editable/develop installs or direct use
try:
    root = Path(__file__).resolve().parents[1]
except NameError:
    root = Path.cwd().resolve().parents[0]
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

from quantlab_data_pipeline import run_ingestion, transform_raw_to_final, run_quality_checks, get_final_data
from quantlab_data_pipeline.logging_utils import configure_logging

configure_logging(level=logging.INFO)


## Run ingestion

Adjust the date range as needed. Ingestion will:
- Pull S&P 500 constituents from WRDS.
- Fetch Alpha Vantage endpoints via MCP.
- Write raw Parquet files under `../data/data-raw/` (relative to repo root).

Note: This can take time and consume API credits. Consider a narrow date window for testing.

In [ ]:
# Example: run a small ingestion window
# Uncomment to execute resume = False
run_ingestion(sleep_seconds=1.0, use_paid_key=True, resume = False)

2025-12-03 16:44:48,763 [INFO] quantlab_data_pipeline.config_loader - Loaded credentials from /Users/edl/Documents/dev/quantlab_v2/quantlab_data_pipeline_api/config/credential.yml
2025-12-03 16:44:48,764 [INFO] quantlab_data_pipeline.ingestion - Using MCP server URL from /Users/edl/Documents/dev/quantlab_v2/quantlab_data_pipeline_api/.vscode/mcp.json: https://mcp.alphavantage.co/mcp?apikey=SB4LTLSNZWSA53UG
2025-12-03 16:44:48,765 [INFO] quantlab_data_pipeline.ingestion - Fetching S&P 500 constituents from WRDS


Loading library list...


2025-12-03 16:44:50,237 [INFO] quantlab_data_pipeline.wrds_client - Querying WRDS for constituents: select permno, mbrstartdt, mbrenddt from crsp_a_indexes.dsp500list_v2
2025-12-03 16:44:50,343 [INFO] quantlab_data_pipeline.wrds_client - Querying WRDS for permno->ticker mapping: select permno, ticker, namedt, nameendt from crsp.msenames


Done


## Transform raw to final

Merges all raw Parquet files into a single long-format table at `../data/final/final_long.parquet`.

In [ ]:
# Uncomment after ingestion to build the final dataset
# final_path = transform_raw_to_final()
# final_path

## Quality checks

Runs basic completeness, consistency, and bounds checks on the final dataset.

In [ ]:
# Uncomment after transform to run checks
# issues = run_quality_checks()
# issues

## Query final data

Load and filter the final dataset using `get_final_data`.

In [ ]:
# Uncomment to sample the final data
# df = get_final_data(tickers=["AAPL", "MSFT"], start_date=date(2024, 1, 1), end_date=date(2024, 1, 5))
# df.head()

In [3]:
# Inspect raw MCP response for a time series call
from quantlab_data_pipeline.alpha_vantage_mcp import AlphaVantageMCPClient
from quantlab_data_pipeline.config_loader import load_credentials
import json

creds = load_credentials()
api_key = creds.get('alphavantage_api_paid') or creds.get('alphavantage_api')
client = AlphaVantageMCPClient(api_key=api_key)
payload = {'symbol': 'IBM', 'datatype': 'csv', 'outputsize': 'full'}
raw = client.call_tool('TIME_SERIES_DAILY_ADJUSTED', payload)

raw[0]


2025-12-03 14:35:20,562 [INFO] quantlab_data_pipeline.config_loader - Loaded credentials from /Users/edl/Documents/dev/quantlab_v2/quantlab_data_pipeline_api/config/credential.yml
2025-12-03 14:35:20,562 [INFO] quantlab_data_pipeline.alpha_vantage_mcp - Calling Alpha Vantage MCP tool TIME_SERIES_DAILY_ADJUSTED


{'type': 'text',
 'text': "{'preview': True, 'data_type': 'csv', 'total_lines': 6564, 'sample_data': 'timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient\\r\\n2025-12-02,307.0,310.4675,301.57,301.78,301.78,4261100,0.0000,1.0\\r\\n2025-12-01,306.505,307.12,302.8,305.67,305.67,3166555,0.0000,1.0\\r\\n2025-11-28,304.06,309.18,303.6,308.58,308.58,1689031,0.0000,1.0\\r\\n2025-11-26,305.18,306.6,301.64,303.21,303.21,2196446,0.0000,1.0\\r\\n2025-11-25,304.125,306.0,297.06,304.48,304.48,2825322,0.0000,1.0\\r\\n2025-11-24,299.18,307.18,297.51,304.12,304.12,6050640,0.0000,1.0\\r\\n2025-11-21,293.48,300.48,291.89,297.44,297.44,5710903,0.0000,1.0\\r\\n2025-11-20,294.64,300.71,290.16,290.4,290.4,5597028,0.0000,1.0\\r\\n2025-11-19,290.5,291.1099,288.07,288.53,288.53,3595912,0.0000,1.0\\r\\n2025-11-18,297.0,297.0,289.92,289.95,289.95,4861928,0.0000,1.0\\r\\n2025-11-17,305.59,306.0,296.51,297.17,297.17,3909741,0.0000,1.0\\r\\n2025-11-14,300.0,307.72,297.59,305.69,305.

In [4]:
raw[0]['text']


"{'preview': True, 'data_type': 'csv', 'total_lines': 6564, 'sample_data': 'timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient\\r\\n2025-12-02,307.0,310.4675,301.57,301.78,301.78,4261100,0.0000,1.0\\r\\n2025-12-01,306.505,307.12,302.8,305.67,305.67,3166555,0.0000,1.0\\r\\n2025-11-28,304.06,309.18,303.6,308.58,308.58,1689031,0.0000,1.0\\r\\n2025-11-26,305.18,306.6,301.64,303.21,303.21,2196446,0.0000,1.0\\r\\n2025-11-25,304.125,306.0,297.06,304.48,304.48,2825322,0.0000,1.0\\r\\n2025-11-24,299.18,307.18,297.51,304.12,304.12,6050640,0.0000,1.0\\r\\n2025-11-21,293.48,300.48,291.89,297.44,297.44,5710903,0.0000,1.0\\r\\n2025-11-20,294.64,300.71,290.16,290.4,290.4,5597028,0.0000,1.0\\r\\n2025-11-19,290.5,291.1099,288.07,288.53,288.53,3595912,0.0000,1.0\\r\\n2025-11-18,297.0,297.0,289.92,289.95,289.95,4861928,0.0000,1.0\\r\\n2025-11-17,305.59,306.0,296.51,297.17,297.17,3909741,0.0000,1.0\\r\\n2025-11-14,300.0,307.72,297.59,305.69,305.69,3592455,0.0000,1.0\\r\\